# POWGEN data reduction

## Introduction

This notebook gives a concise overview of how to use the ESSDiffraction package with Sciline.
It uses a simple reduction workflow for the SNS [POWGEN](https://sns.gov/powgen) experiment.

We begin with relevant imports:

In [ ]:
import scipp as sc
import scippneutron as scn
import sciline

from ess import powder
from ess.powder.external import powgen
from ess.powder.types import *

## Define reduction parameters

We define a dictionary containing the reduction parameters.
The keys are types defined in [essdiffraction.types](../generated/modules/ess.diffraction.types.rst).

In [ ]:
params = {
    NeXusDetectorName: "powgen_detector",
    # Input data
    Filename[SampleRun]: powgen.data.powgen_tutorial_sample_file(),
    Filename[VanadiumRun]: powgen.data.powgen_tutorial_vanadium_file(),
    CalibrationFilename: powgen.data.powgen_tutorial_calibration_file(),
    # The upper bounds mode is not yet implemented.
    UncertaintyBroadcastMode: UncertaintyBroadcastMode.drop,
    # Edges for binning in d-spacing
    DspacingBins: sc.linspace("dspacing", 0.0, 2.3434, 201, unit="angstrom"),
    # Mask in time-of-flight to crop to valid range
    TofMask: lambda x: (x < sc.scalar(0.0, unit="us"))
    | (x > sc.scalar(16666.67, unit="us")),
    TwoThetaMask: None,
    WavelengthMask: None,
}

## Create pipeline using Sciline

We use the basic providers available in `essdiffraction` as well as the specialised `powder` and `powgen` providers.

In [ ]:
providers = [*powder.providers, *powgen.providers]
pipeline = sciline.Pipeline(providers, params=params)
pipeline = powder.with_pixel_mask_filenames(pipeline, [])

## Use the pipeline

### Compute final result

We can get the graph for computing the final intensity as a function of d-spacing:

In [ ]:
pipeline.visualize(IofDspacing, graph_attr={"rankdir": "LR"})

Now we compute the result:

In [ ]:
result = pipeline.compute(IofDspacing)
result

In [ ]:
dspacing_histogram = result.hist()
dspacing_histogram.plot()

### Save reduced data to file

We ultimately need to write the reduced data to a file.
This could be done with the `result` we computed above.
But we can use the pipeline to provide additional parameters (in this case only the file name) as shown below.
See also the [File output](https://scipp.github.io/sciline/recipes/recipes.html#File-output) docs of Sciline.

For simplicity we write a simply xye file with 3 columns: $d$-spacing, intensity, standard deviation of intensity.

In [ ]:
def save_xye(
    reduced_data: IofDspacing,
    out_filename: OutFilename,
) -> None:
    data = reduced_data.hist()
    data.coords["dspacing"] = sc.midpoints(data.coords["dspacing"])
    scn.io.save_xye(out_filename, data, coord="dspacing")

Insert a new parameter to set the file name.
This could have been done at the top where the other parameters are defined.

In [ ]:
pipeline[OutFilename] = "reduced.xye"

And use the pipeline to write the file.
Note that this recomputes the result!

In [ ]:
pipeline.bind_and_call(save_xye)

### Compute intermediate results

For inspection and debugging purposes, we can also compute intermediate results.
To avoid repeated computation (including costly loading of files), we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
results = pipeline.compute(
    (
        ReducibleDetectorData[SampleRun],
        MaskedData[SampleRun],
        FilteredData[SampleRun],
        FilteredData[VanadiumRun],
    )
)

In [ ]:
results[ReducibleDetectorData[SampleRun]]

In [ ]:
results[MaskedData[SampleRun]].bins.concat().hist(wavelength=300).plot()

In [ ]:
tof_data = sc.DataGroup(
    sample=results[FilteredData[SampleRun]].bins.concat(),
    vanadium=results[FilteredData[VanadiumRun]].bins.concat(),
)
tof_data.hist(tof=100).plot()

## Group by scattering angle

The above pipeline focuses the data by merging all instrument pixels to produce a 1d d-spacing curve.
If instead we want to group into $2\theta$ bins, we can alter the pipeline parameters by adding some binning in $2\theta$:

In [ ]:
pipeline[TwoThetaBins] = sc.linspace(
    dim="two_theta", unit="deg", start=25.0, stop=90.0, num=17
).to(unit="rad")

We then have to request a final result that depends on both d-spacing and $2\theta$:

In [ ]:
pipeline.visualize(IofDspacingTwoTheta, graph_attr={"rankdir": "LR"})

Compute and plot the result:

In [ ]:
grouped_dspacing = pipeline.compute(IofDspacingTwoTheta)
grouped_dspacing

In [ ]:
angle = sc.midpoints(grouped_dspacing.coords["two_theta"])
sc.plot(
    {
        f"{angle[group].value:.3f} {angle[group].unit}": grouped_dspacing[
            "two_theta", group
        ].hist()
        for group in range(2, 6)
    }
)

Or we can view it as a 2D plot, which should display powder peaks as vertical bright lines:

In [ ]:
grouped_dspacing.hist().plot()